In [10]:
import numpy as np
from copy import deepcopy

# https://inspiredacoustics.com/en/MIDI_note_numbers_and_center_frequencies
# Define scales commonly used in techno music (MIDI note values)
scales = {
    "C Minor": [60, 62, 63, 65, 67, 68, 70, 72, 74, 75, 77, 79, 80, 82, 84],
    "G Minor": [55, 57, 58, 60, 62, 63, 65, 67, 69, 70, 72, 74, 75, 77, 79],
    "F Minor": [53, 55, 56, 58, 60, 61, 63, 65, 67, 68, 70, 72, 73, 75, 77],
    "A Minor": [57, 59, 60, 62, 64, 65, 67, 69, 71, 72, 74, 76, 77, 79],
}

# https://blog.native-instruments.com/edm-chord-progressions/
# Common chord progressions in techno music one chord followed by another chord followed
# by another chord, etc. A Chord consists of some number of different notes being played
# at the same time
chord_progressions = {
    "i-VII-VI-VII": [1, 7, 6, 7],
    "i-III-VII-i": [1, 3, 7, 1],
    "i-VI-VII-i": [1, 6, 7, 1],
    "i-III": [1, 3, 1, 3],
}


class problem:
    def __init__(self, scale_name="C Minor", progression_name="i-VII-VI-VII"):
        self.number_of_genes = 32 # Length of our melody
        self.scale = scales[scale_name]
        self.scale_name = scale_name
        self.min_value = min(self.scale)
        self.max_value = max(self.scale)
        self.progression = chord_progressions[progression_name]
        self.progression_name = progression_name
        self.cost_function = self.melody_cost
        self.acceptable_cost = 30

        # Techno rhythm pattern (1 = note, 0 = rest) - 16th note grid
        self.rhythm_pattern = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0] * 2

        # Derive the root note of the scale
        self.root_note = self.scale[0] % 12

        # Calculate scale degrees (0-based within an octave)
        self.scale_degrees = [note % 12 for note in self.scale]

        # Map progression to actual chord roots
        self.progression_roots = []
        root_index = self.scale_degrees.index(self.root_note)
        for degree in self.progression:
            # Convert 1-based to 0-based
            index = (root_index + degree - 1) % len(self.scale_degrees)
            self.progression_roots.append(self.scale_degrees[index])

    def melody_cost(self, chromosome):
        # Calc scale adherence penalty
        scale_cost = 0
        for note in chromosome:
            # Get closest note thats actually in the scale
            closest_scale_note = min(self.scale, key=lambda x: abs(x - note))
            # Add the distance to the penalty, bigger distance = bigger penalty
            scale_cost += abs(note - closest_scale_note)

        # Weight factors for techno
        scale_weight = 10.0
        jump_weight = 0.7        # Techno can handle some jumps
        interval_weight = 0.4    # Techno often uses simple intervals
        harmony_weight = 3.0     # Techno uses simpler harmony
        repetition_weight = 5.0  # Techno likes to repeat itself

        total_cost = scale_cost * scale_weight

        # Evaluate melodic movement between consecutive notes
        interval_cost = 0
        for i in range(len(chromosome) - 1):
            # How far apart are adjacent notes
            interval = abs(chromosome[i] - chromosome[i + 1])

            # Penalize jumps larger than a perfect fifth (7 semitones)
            # Techno can handle some jumps but too many large ones sound jarring
            if interval > 7:
                interval_cost += (interval - 7) * jump_weight

            # Penalize weird intervals that dont typically sound good in techno
            # prefers same note (0) whole step (2) minor third (3) fourth (5)
            # fifth (7) or octave (12)
            consonant_intervals = [0, 2, 3, 5, 7, 12]
            if (interval % 12) not in consonant_intervals:
                interval_cost += 1

        total_cost += interval_cost * interval_weight

        # Check how well the melody fits with chord progression
        harmony_cost = 0
        # Split melody into chunks that match our chord progresion
        segment_length = len(chromosome) // len(self.progression)

        for i, chord_root in enumerate(self.progression_roots):
            start_idx = i * segment_length
            end_idx = start_idx + segment_length
            segment = chromosome[start_idx:end_idx]

            for note in segment:
                # Get just the note name w/out octave (C C# D etc)
                note_class = note % 12
                # Figure out which notes make up current chord
                # For techno using minor chords with root minor 3rd and perfect 5th
                chord_tones = [
                    chord_root,
                    (chord_root + 3) % 12,
                    (chord_root + 7) % 12
                ]

                # If the note isnt part of the chord add some penalty
                if note_class not in chord_tones:
                    # Go easier on notes that atleast belong to the scale
                    if note_class in self.scale_degrees:
                        harmony_cost += 0.5  # Small penalty - techno can be more modal
                    else:
                        harmony_cost += 2    # Bigger penalty for totally out-of-scale notes

        total_cost += harmony_cost * harmony_weight

        repetition_cost = 0

        # Look for repeating 4-note patterns super common in techno
        pattern_length = 4

        if len(chromosome) > pattern_length * 2:
            # Count how many times patterns repeat in the melody
            pattern_matches = 0

            for i in range(0, len(chromosome) - pattern_length, pattern_length):
                # Get a pattern from the melody
                pattern = chromosome[i:i+pattern_length]
                # See if this pattern shows up again later
                for j in range(i + pattern_length, len(chromosome) - pattern_length + 1, pattern_length):
                    comparison = chromosome[j:j+pattern_length]
                    if pattern == comparison:
                        pattern_matches += 1
                        break

            # Figure out how far we are from ideal repitition
            # Good techno has lots of repetition but not to much
            ideal_matches = len(chromosome) // (pattern_length * 2)
            repetition_cost = abs(pattern_matches - ideal_matches) * 2

        total_cost += repetition_cost * repetition_weight

        # Make sure melody ends cleanly
        # In techno ending on the root note or the fifth sounds best
        if chromosome[-1] % 12 not in [self.root_note, (self.root_note + 7) % 12]:
            total_cost += 5  # Penalty for wierd ending notes

        return total_cost


class individual:
    def __init__(self, prob):
        # Create random chromosome - just use random notes from the scale
        self.chromosome = [np.random.choice(prob.scale) for _ in range(prob.number_of_genes)]
        self.cost = prob.cost_function(self.chromosome)

    def mutate(self, rate_of_gene_mutation, range_of_gene_mutation, prob):
        for index in range(len(self.chromosome)):
            if np.random.uniform() < rate_of_gene_mutation:
                # For techno, we need smarter mutations
                mutation_type = np.random.choice(["change_note", "repeat_pattern", "octave_shift"])

                if mutation_type == "change_note":
                    # 80% chance to pick a note from the scale
                    if np.random.uniform() < 0.8:
                        self.chromosome[index] = np.random.choice(prob.scale)
                    else:
                        # 20% chance to shift slightly
                        self.chromosome[index] += np.random.randn() * range_of_gene_mutation

                elif mutation_type == "repeat_pattern" and index + 4 < len(self.chromosome):
                    # Repeat a small pattern
                    if index >= 4:
                        # Copy previous 4 notes
                        source = self.chromosome[index-4:index]
                        self.chromosome[index:index+4] = source

                elif mutation_type == "octave_shift":
                    # Shift by an octave
                    self.chromosome[index] += 12 if np.random.uniform() < 0.5 else -12

    def crossover(self, parent2, explore_crossover, prob=None):
        # For melody, I want to preserve patterns of 4 notes (common in techno)
        child1 = deepcopy(self)
        child2 = deepcopy(parent2)

        if np.random.uniform() < 0.7:  # 70% pattern-based crossover
            # Exchange 4-note patterns between parents
            pattern_size = 4

            for i in range(0, len(self.chromosome), pattern_size):
                end = min(i + pattern_size, len(self.chromosome))
                if np.random.uniform() < 0.5:  # 50% chance to swap each note
                    child1.chromosome[i:end] = parent2.chromosome[i:end]
                    child2.chromosome[i:end] = self.chromosome[i:end]
        else:
            # Same crossover method
            alpha = np.random.uniform(-explore_crossover, 1+explore_crossover)
            for i in range(len(self.chromosome)):
                child1.chromosome[i] = alpha * self.chromosome[i] + (1-alpha) * parent2.chromosome[i]
                child2.chromosome[i] = alpha * parent2.chromosome[i] + (1-alpha) * self.chromosome[i]
                # round to closest integer as using MIDI values
                child1.chromosome[i] = round(child1.chromosome[i])
                child2.chromosome[i] = round(child2.chromosome[i])

        return child1, child2


#parameters
class parameters:
    def __init__(self):
        self.population = 175  # number of melodies
        self.number_of_generations = 100  # for how many generatios are we running
        self.child_rate_per_generation = 0.5  # How many new childreen on each generation?
        self.crossover_explore_rate = 0.2  # Exploration rate in crossovers
        self.gene_mutate_range = 2.0  # When mutating, change by at most 2 semitones
        self.gene_mutate_rate = 0.4  # Chance of mutating each gene?


# Run Genetic method here
def choose_indices_from(number_in_list):
    index_1 = np.random.randint(number_in_list)
    index_2 = np.random.randint(number_in_list)
    if index_1 == index_2:
        return choose_indices_from(number_in_list)
    return index_1, index_2


# Genetic algorithm implementation
def run_genetic(prob, params):
    # Read in important info from problem
    cost_function = prob.cost_function

    number_in_population = params.population
    max_number_of_generations = params.number_of_generations
    number_of_children_per_generation = int(params.child_rate_per_generation * number_in_population)
    explore_crossover = params.crossover_explore_rate
    gene_mutate_rate = params.gene_mutate_rate
    gene_mutate_range = params.gene_mutate_range

    # Generate initial population
    population = []

    # Placeholder for best solution
    best_solution = individual(prob)
    best_solution.cost = np.inf

    for i in range(number_in_population):
        new_individual = individual(prob)
        population.append(new_individual)

        if new_individual.cost < best_solution.cost:
            best_solution = deepcopy(new_individual)

    best_costs = []
    avg_costs = []

    for iteration in range(max_number_of_generations):
        children = []

        while len(children) < number_of_children_per_generation:
            parent1_index, parent2_index = choose_indices_from(len(population))
            parent1 = population[parent1_index]
            parent2 = population[parent2_index]

            child1, child2 = parent1.crossover(parent2, explore_crossover, prob)
            child1.mutate(gene_mutate_rate, gene_mutate_range, prob)
            child1.cost = cost_function(child1.chromosome)
            child2.mutate(gene_mutate_rate, gene_mutate_range, prob)
            child2.cost = cost_function(child2.chromosome)

            children.append(child1)
            children.append(child2)

        # Add the children to the population
        population.extend(children)

        # Sort the population by cost
        population.sort(key=lambda x: x.cost)

        # Keep only the best individuals
        population = population[:number_in_population]

        # Update best solution if better found
        if population[0].cost < best_solution.cost:
            best_solution = deepcopy(population[0])

        current_costs = [ind.cost for ind in population]
        best_costs.append(min(current_costs))
        avg_costs.append(sum(current_costs) / len(current_costs))

        # Print progress every 10 generations for cleaner output
        if iteration % 10 == 0:
            print(f"Generation {iteration}: Best cost = {best_solution.cost:.2f}")

    print(f"Evolution complete! Final best cost: {best_solution.cost:.2f}")
    return best_solution, best_costs, avg_costs


#  Running of the algorithm with outputs here
# Function to play the melody (doesn't work in Colab, I used Pycharm and installed Pygame inside there)
def play_melody(chromosome, scale_name="C Minor", tempo=130):
    print(f"Melody (in {scale_name}, tempo={tempo}): {chromosome}")
    try:
        import pygame.midi
        pygame.midi.init()
        player = pygame.midi.Output(0)

        # Calculate note duration in ms based on tempo (in BPM)
        duration_ms = 60000 / tempo

        # Play each note
        for note in chromosome:
            player.note_on(note, 100)  # note, velocity
            pygame.time.wait(int(duration_ms/4))  # 16th notes for techno
            player.note_off(note, 100)

        pygame.midi.quit()
        return True
    except:
        print("Could not play melody. Make sure pygame is installed.")
        return False


if __name__ == "__main__":
    # Create problem instance
    p1 = problem(scale_name="C Minor", progression_name="i-VII-VI-VII")

    par1 = parameters()

    best_melody, best_costs, avg_costs = run_genetic(p1, par1)

    # Play the resulting melody
    play_melody(best_melody.chromosome, p1.scale_name)

Generation 0: Best cost = 81.62
Generation 10: Best cost = 56.68
Generation 20: Best cost = 56.68
Generation 30: Best cost = 56.68
Generation 40: Best cost = 56.68
Generation 50: Best cost = 56.68
Generation 60: Best cost = 56.68
Generation 70: Best cost = 56.68
Generation 80: Best cost = 56.68
Generation 90: Best cost = 36.68
Evolution complete! Final best cost: 36.68
Melody (in C Minor, tempo=130): [np.int64(70), np.int64(74), np.int64(80), np.int64(70), np.int64(75), np.int64(72), np.int64(79), np.int64(70), np.int64(75), np.int64(72), np.int64(79), np.int64(70), np.int64(75), np.int64(70), np.int64(63), np.int64(70), np.int64(75), np.int64(70), np.int64(63), np.int64(70), np.int64(75), np.int64(70), np.int64(63), np.int64(70), np.int64(75), np.int64(70), np.int64(63), np.int64(70), np.int64(82), np.int64(82), np.int64(63), np.int64(72)]
Could not play melody. Make sure pygame is installed.
